In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
from evaluate import load
import re
import pandas as pd
import warnings 
warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
splits = {
    'train': 'data/train-00000-of-00001-87e767a83d108945.parquet',
    'validation': 'data/validation-00000-of-00001-2d1ce84ca498cf0b.parquet'
}
dataset = pd.read_parquet("hf://datasets/musabg/wikipedia-tr-summarization/" + splits["train"])

In [ ]:
reviews_sample = dataset.sample(n=4000, random_state=42)

reviews_sample = reviews_sample.rename(columns={"text": "article", "summary": "summary"})

def clean_text(text):
    text = re.sub(r"[^\w\s.,!?ğüşöçıİĞÜŞÖÇ]", "", text)  
    text = re.sub(r"\s+", " ", text).strip()  
    return text

reviews_sample["article"] = reviews_sample["article"].apply(clean_text)

In [ ]:
hf_dataset = Dataset.from_pandas(reviews_sample)

train_test_split = hf_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

In [ ]:
model_name = "ozcangundes/mt5-small-turkish-summarization"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]  
    outputs = examples["summary"]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=200, truncation=True, padding="max_length")  
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(preprocess_data, batched=True, remove_columns=val_dataset.column_names)

In [ ]:
rouge = load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    if isinstance(result["rouge1"], dict):
        return {
            "rouge1": result["rouge1"]["fmeasure"] * 100,
            "rouge2": result["rouge2"]["fmeasure"] * 100,
            "rougeL": result["rougeL"]["fmeasure"] * 100,
        }
    return {
        "rouge1": result["rouge1"] * 100,
        "rouge2": result["rouge2"] * 100,
        "rougeL": result["rougeL"] * 100,
    }

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch", 
    save_strategy="epoch",  
    save_total_limit=3,  
    learning_rate=2e-5,
    lr_scheduler_type="linear",  
    warmup_steps=500,
    per_device_train_batch_size=6, 
    per_device_eval_batch_size=6,
    num_train_epochs=4,  
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    metric_for_best_model="eval_loss",
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics 
)


In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./trained_model")
tokenizer.save_pretrained("./trained_model")

In [8]:
from transformers import pipeline
import re

summarizer = pipeline("summarization", model="./trained_model", tokenizer="./trained_model")

def summarize_text_in_chunks(text, chunk_size=3, max_length=90, min_length=30):
    """
    Metni belirli bir sayıda cümleye bölerek özetler.
    
    Args:
        text (str): Özetlenecek metin.
        chunk_size (int): Her bir parçada kaç cümle olacağı.
        max_length (int): Özetin maksimum uzunluğu (token sayısı).
        min_length (int): Özetin minimum uzunluğu (token sayısı).
        
    Returns:
        str: Parçalanmış ve özetlenmiş metinlerin birleşimi.
    """
    # Metni cümlelere böl
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-ZİĞÜŞÖÇ][a-zığüşöç]\.)(?<=\.|\?)\s', text)
    
    # Cümleleri gruplara ayır
    chunks = [' '.join(sentences[i:i+chunk_size]) for i in range(0, len(sentences), chunk_size)]
    
    # Her bir parçayı özetle
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)[0]["summary_text"]
        summaries.append(summary)
    
    # Tekrar eden kelimeleri temizle
    combined_summary = " ".join(summaries)
    combined_summary = clean_redundancies(combined_summary)
    
    return combined_summary

def clean_redundancies(text):
    """
    Metindeki tekrar eden ifadeleri temizler ve anlamsız tekrarları kaldırır.
    
    Args:
        text (str): Temizlenecek metin.
        
    Returns:
        str: Temizlenmiş metin.
    """
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-ZİĞÜŞÖÇ][a-zığüşöç]\.)(?<=\.|\?)\s', text)
    seen_sentences = set()
    cleaned_sentences = []
    
    for sentence in sentences:
        cleaned_sentence = remove_repeated_phrases(sentence)
        if cleaned_sentence.lower() not in seen_sentences:
            seen_sentences.add(cleaned_sentence.lower())
            cleaned_sentences.append(cleaned_sentence)
    
    return " ".join(cleaned_sentences)

def remove_repeated_phrases(sentence):
    """
    Bir cümledeki tekrar eden kelime öbeklerini temizler.
    
    Args:
        sentence (str): Temizlenecek cümle.
        
    Returns:
        str: Tekrarlar temizlenmiş cümle.
    """
    words = sentence.split()
    seen_words = set()
    cleaned_words = []
    
    for word in words:
        if word not in seen_words:
            seen_words.add(word)
            cleaned_words.append(word)
    
    return " ".join(cleaned_words)

def post_process_summary(summary):
    """
    Özet sonrası işleme fonksiyonu.
    
    Args:
        summary (str): İşlenecek özet.
        
    Returns:
        str: İşlenmiş özet.
    """
    summary = summary.strip()
    summary = summary.capitalize()
    summary = re.sub(r" +", " ", summary)  
    summary = re.sub(r"(?<=[.,])(?=[^\s])", " ", summary)  
    return summary

article = """Anderson-Darling sınaması, istatistik bilim dalında, bir parametrik olmayan istatistik sınaması 
olup örneklem verilerinin belirli bir olasılık dağılımı gösterip göstermediğini sınamak için, yani uygunluk 
iyiliği sınaması için, kullanılmaktadır. Bu sınama ilk defa 1952'de Amerikan istatistikçileri T.W.Anderson Jr. 
ile D.A.Darling tarafından yayınlanmıştır. Hem çok küçük (n le; 25) örneklem sayılı veriler için hem de hacmi 
200u aşan sanayi kalite kontrol verileri için başarıyla normallik sınaması için kullanıldığı bildirilmiştir. 
Genel uygunluk iyiliği sınaması Anderson-Darling sınaması bir örneklem verisinin tam olarak belirlenmiş bir olasılık 
dağılımı gösteren bir anakütleden gelip gelmediğinin sınanması için kullanılır. Verilmiş N büyüklük sayıda bir örneklem 
veri serisi, yani , kullanılır. Bu serinin sınanmansı için hangi olasılık dağılımından geldiğinin ve bu olasılık dağılımını 
tam olarak belirleyen parametre değerinin veya parametreler değerlerinin verilmesi gerekir.Anderson-Darling sınaması için sıfır 
hipotez, her türden uygunluk iyiliği sınaması gibi, örneklem verilerin için tüm parametre değerleri ile iyice belirlenen 
olasılık dağılımlı anakütleden geldiğidir. Bu sıfır hipotezin çok sınırlı olduğuna dikkat çekilmelidir. Ancak verilmiş 
parametre veya parametreler için olasılık dağılımı uygulanması incelenmektedir. Eğer sıfır hipotez sınama sonucu ret 
edilirse, verilerin parametre(ler) ile belirlenmiş dağılıma uymadığı sonucuna varılır. Tekrar edilmelidir ki genel 
olarak belli bir dağılım ret edilmemektedir; sadece belli parametresi olan dağılım ret edilmektedir.Elde edilen veriler 
en küçük değerden en büyük değere kadar bir sıraya konulur. Bu sıraya konulmuş veriler, yani , bir sınama istatistiğinin 
hesaplanması için kullanılır. Parametresi veya parametreleri verilmiş olasılık dağılımı için birikimli dağılım 
fonksiyonu kullanılarak bir sıra değerleri bulununur. Bu iki seri kullanılarak önce şu S toplamı elde edilir. Bu toplam 
kullanılarak Anderson-Darling istatistiği değeri yani elde edilir.Sıfır hipotezde belirtilen olasılık dağılımına göre, 
elde edilen değerinin belirli bir sabitle (çok kere örneklem hacmi 'N'e bağlı olarak) çarpılması gerektir ve bu değiştirilmiş 
Anderson-Darling istatistiği adı altında sınama istatistiği olarak kullanılır. sınama istatistiği belirlenen teorik 
olasılık dağılımı için p-değeri bulmak için kullanılır. Hesaplanmış p-değeri eğer %1 veya %5 olan anlamlılık seviyesinden 
büyük ise sıfır hipotez kabul edilir ve örneklem verisi belirlenen olasılık dağılımına uyduğu sonucuna varılır. Ancak bu 
p-değeri bulma işlemi bir olasılık dağılımı simülasyonu gerekeceği için bilgisayarla sayısal hesaplama gerektirir.Bazı 
olasılık dağılımları için özel tablolar geliştirilmiş ve değişik parametre değerleri ve belirtilmiş anlamlılık değerleri 
için (genellikle %1 ve %5) kritik değerler tabloda belirtilmiştir. Normal dağılım, log-normal dağılım, üstel dağılım, 
Weibull dağılımı, logistik dağılım ve Tip I uçsal değerler için bu tabloların bulunduğu bilinmektedir. Tablodan 
bulunan kritik değer, hesaplanmış değeri ile karşılaştırılır. Belirlenmiş olasılık dağılımına uygunluk sıfır hipotezinin 
kabul edilmesi sonucudur yani hesaplanmış değer tablo kritik değerinden büyükse örneklem verileri belirlenmiş olasılık 
dağılımına uygunluk gösterir sonucuna varılır."""

final_summary = summarize_text_in_chunks(article)
processed_summary = post_process_summary(final_summary)
print("Makale Özeti:", processed_summary)

Device set to use cpu
Your max_length is set to 90, but your input_length is only 74. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)
Your max_length is set to 90, but your input_length is only 59. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


Makale Özeti: Anderson-darling sınaması, istatistik bilim dalında, bir parametrik olmayan sınaması olup örneklem verilerinin belirli olasılık dağılımı gösterip göstermediğini sınamak için, yani uygunluk iyiliği için kullanılmaktadır. verilmiş n büyüklük sayıda bir örneklem veri serisi, yani , kullanılır ve olasılık dağılımından geldiğinin bu dağılımını tam olarak belirleyen parametre değerinin veya parametreler değerlerinin verilmesi gerekir özellikle dağılımı uygulanması verilerin sonucu verilmiş için incelenmektedir. geçtiğimiz günlerde sınama isteniğinin hesaplanması için kullanılan bir sıra değerleri bulunan değeri bulunur ve kullanılır. anderson-darling istatistiği değeri yani elde edilen , sınama olarak kullanılır ve adı altında belirlenen teorik olasılık dağılımı için p-değeri bulmak kullanılır. olasılık dağılımı simülasyonu için bilgisayarla sayısal hesaplama gerektirir ve gerekeceği gerektirir. belirlenmiş olasılık dağılımına uygunluk sıfır hipotezinin kabul edilmesi sonucudur

In [9]:
from transformers import pipeline
import re
import warnings 
warnings.simplefilter(action='ignore', category=Warning)

# Eğitilen modeli yükleme
summarizer = pipeline("summarization", model="./trained_model", tokenizer="./trained_model")

def summarize_text_in_chunks(text, chunk_size=3, max_length=90, min_length=30):
    """
    Metni belirli bir sayıda cümleye bölerek özetler.
    
    Args:
        text (str): Özetlenecek metin.
        chunk_size (int): Her bir parçada kaç cümle olacağı.
        max_length (int): Özetin maksimum uzunluğu (token sayısı).
        min_length (int): Özetin minimum uzunluğu (token sayısı).
        
    Returns:
        str: Parçalanmış ve özetlenmiş metinlerin birleşimi.
    """
    # Metni cümlelere böl
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-ZİĞÜŞÖÇ][a-zığüşöç]\.)(?<=\.|\?)\s', text)
    
    # Cümleleri gruplara ayır
    chunks = [' '.join(sentences[i:i+chunk_size]) for i in range(0, len(sentences), chunk_size)]
    
    # Her bir parçayı özetle
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)[0]["summary_text"]
        summaries.append(summary)
    
    # Tekrar eden kelimeleri temizle
    combined_summary = " ".join(summaries)
    combined_summary = clean_redundancies(combined_summary)
    
    return combined_summary

def clean_redundancies(text):
    """
    Metindeki tekrar eden ifadeleri temizler ve anlamsız tekrarları kaldırır.
    
    Args:
        text (str): Temizlenecek metin.
        
    Returns:
        str: Temizlenmiş metin.
    """
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-ZİĞÜŞÖÇ][a-zığüşöç]\.)(?<=\.|\?)\s', text)
    seen_sentences = set()
    cleaned_sentences = []
    
    for sentence in sentences:
        cleaned_sentence = remove_repeated_phrases(sentence)
        if cleaned_sentence.lower() not in seen_sentences:
            seen_sentences.add(cleaned_sentence.lower())
            cleaned_sentences.append(cleaned_sentence)
    
    return " ".join(cleaned_sentences)

def remove_repeated_phrases(sentence):
    """
    Bir cümledeki tekrar eden kelime öbeklerini temizler.
    
    Args:
        sentence (str): Temizlenecek cümle.
        
    Returns:
        str: Tekrarlar temizlenmiş cümle.
    """
    words = sentence.split()
    seen_words = set()
    cleaned_words = []
    
    for word in words:
        if word not in seen_words:
            seen_words.add(word)
            cleaned_words.append(word)
    
    return " ".join(cleaned_words)

def post_process_summary(summary):
    """
    Özet sonrası işleme fonksiyonu.
    
    Args:
        summary (str): İşlenecek özet.
        
    Returns:
        str: İşlenmiş özet.
    """
    summary = summary.strip()
    summary = summary.capitalize()
    summary = re.sub(r" +", " ", summary) 
    summary = re.sub(r"(?<=[.,])(?=[^\s])", " ", summary)  
    return summary

article = """Ankara (/ˈaŋkaɾa/), Türkiye'nin başkenti ve Ankara ilinin merkezi olan şehirdir.[3] 
Coğrafi olarak Türkiye'nin merkezine yakın bir konumda bulunur ve İç Anadolu Bölgesi'nde yer alır.
Ankara'nın 13 Ekim 1923'te[4] başkent ilan edilmesinin ardından şehir hızla gelişmiş ve Türkiye'nin 
ikinci en kalabalık ili olmuştur. Türkiye Cumhuriyeti'nin ilk yıllarında ekonomisi tarım ve 
hayvancılığa dayanan ilin topraklarının yarısı hâlâ tarım amaçlı kullanılmaktadır. Ekonomik etkinlik 
büyük oranda ticaret ve sanayiye dayalıdır. Tarım ve hayvancılığın ağırlığı ise giderek azalmaktadır. 
Ankara ve civarındaki gerek kamu sektörü gerek özel sektör yatırımları, başka illerden büyük bir nüfus 
göçünü teşvik etmiştir. Cumhuriyetin kuruluşundan günümüze, nüfusu ülke nüfusunun iki katı hızda artmıştır. 
Nüfusun yaklaşık dörtte üçü hizmet sektörü olarak tanımlanabilecek memuriyet, ulaşım, haberleşme ve 
ticaret benzeri işlerde, dörtte biri sanayide, %2'si ise tarım alanında çalışır. Sanayi, özellikle 
tekstil, gıda ve inşaat sektörlerinde yoğunlaşmıştır. Günümüzde ise en çok savunma, metal ve motor 
sektörlerinde yatırım yapılmaktadır. Türkiye'nin en çok sayıda üniversiteye sahip ikinci ili olan[5]
Ankara'da ayrıca, üniversite diplomalı kişi oranı ülke ortalamasının iki katıdır. Bu eğitimli nüfus, 
teknoloji ağırlıklı yatırımların gereksinim duyduğu iş gücünü oluşturur. Ankara'dan otoyollar, 
demiryolu ve havayoluyla Türkiye'nin diğer şehirlerine ulaşılır.
Bilinen tarihi en az 10 bin yıl öncesine, Eski Taş Çağı'na ulaşan[6] Ankara, tarih öncesinden 
günümüze dek pek çok medeniyeti barındırmıştır. Hititler, Frigyalılar, Lidyalılar, Persler, 
Galatlar, Romalılar, Bizanslılar, Selçuklular, Osmanlılar ve nihayet Türkiye Cumhuriyeti, il 
topraklarını kontrolleri altında tutmuştur. Tektosagların ve Türkiye Cumhuriyeti'nin başkenti 
olan Ankara şehri ve Frigyalıların başkenti Gordion, il sınırları içinde yer alır. Yıldırım 
Bayezid'in Timur'a yenik düştüğü Ankara Muharebesi Çubuk yakınlarında ve Türk Kurtuluş Savaşı'nın 
dönüm noktası olan Sakarya Muharebesi ise Polatlı yakınlarında yapılmıştır."""

final_summary = summarize_text_in_chunks(article)
processed_summary = post_process_summary(final_summary)
print("Makale Özeti:", processed_summary)


Device set to use cpu
Your max_length is set to 90, but your input_length is only 67. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)
Your max_length is set to 90, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 90, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


Makale Özeti: Ankara, ankara'nın başkenti ve ankara ilinin merkezi olan şehirdir i̇ç anadolu bölgesi'nde yer alır alır. ankara ve civarındaki gerek kamu sektörü özel sektör yatırımları, başka illerden büyük bir nüfus göçünü teşvik etmiştir. cumhuriyetin kuruluşundan günümüze, nüfusu ülke nüfusunun iki katı hızda artmıştır ve artmıştır. türkiye'nin en çok savunma, metal ve motor sektörlerinde yatırım yapılmaktadır. türkiye'nin en çok sayıda üniversiteye sahip ikinci ili olan ve ankara'da ayrıca, üniversite diplomalı kişi oranı ülke ortalamasının iki katıdır. ankara'dan otoyollar, demiryolu ve havayoluyla türkiye'nin diğer şehirlerine ulaşılır tarih öncesi en az 10 bin yıl öncesine, eski taş çağı'na ulaşan öncesine göre tarihi ulaşmıştır. tektosagların ve türkiye cumhuriyeti'nin başkenti gordion, il sınırları içinde yer alır türk kurtuluş savaşı'nın dönüm noktası olan sakarya muharebesi ise polatlı yakınlarında yapılmıştır.
